In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np
from matplotlib import pyplot as plt


pd.options.display.float_format = "{:,.2f}".format
sns.set_theme(style='whitegrid')

# Config

In [2]:
PATH_DATASET = 'data/banco_carnes_un_v9_ed.parquet'

PATH_CORRECTION = 'data/banco_correcao.csv'

CODES = {
    11: 'CARNE BOVINA',
    12: 'CARNE SUINA',
    13: 'OUTRAS CARNES',
    14: 'AVES',
    15: 'CARNES SECAS E SALGADAS',
    16: 'VÍSCERAS',
    #17: 'PREPARAÇÕES QUE INCLUEM CARNES BOVINAS',
    #18: 'PREPARAÇÕES QUE INCLUEM OUTRAS CARNES',
    21: 'ALMÔNDEGAS, HAMBURGERS E NUGGETS',
    22: 'EMBUTIDOS, FRIOS E CONSERVAS',
    #23: 'PREPARAÇÕES QUE INCLUEM CARNES UP',	
    31: 'PESCADOS FRESCOS',
    32: 'PESCADOS EM CONSERVA',
    33: 'PESCADOS SALGADOS',
    #34: 'PREPARAÇÕES QUE INCLUEM PESCADOS',
    41: 'OVOS INTEIROS',
    #42: 'REFEIÇÕES QUE INCLUEM OVOS',
    51: 'PTS - CARNE DE SOJA',
    #52: 'PREPARAÇÕES QUE INCLUEM CARNE DE SOJA'
}


VIS_FIELDS = [
    'codigo',
    #'ano',
	'item',
	#'un',
	'qt',
    'qt_origin',
	'valor_un',
	'valor',
	#'valor_real,
	'un_adj',
    'price_kg',
    'rule_id',
    'unit_kg',
    'outlier',
    'median_price'
    #'rule_version',
    #'rule_id'
    #'is_outlier_qt',
    #'quartile_range_valor_un',
    #'quartile_range_qt'
]


# check 1% da parte superior da variável qt

# Input

In [3]:
df = pd.read_parquet(PATH_DATASET)

df_correction = pd.read_csv(PATH_CORRECTION)

# Fix Records
It applies special rules defined by the team to adjust 50 records based on the field "correcao"

In [4]:
# apply rule median
median_1 = df.query('item.str.contains("SALSICHA, DIVERSOS SABORES, COM OU SEM MOLHO") and ano == 2017')['valor_un'].median()
median_2 = df.query('item == "SALSICHA EM CONSERVA" and ano == 2017')['valor_un'].median()

df = df.set_index(['ano', 'item', 'dados.CNPJ.CPF', '...1', 'valor_real'])
df_correction = df_correction.set_index(['ano', 'item', 'dados.CNPJ.CPF', '...1', 'valor_real'])

df.loc[
    df.index.isin(df_correction.query('correcao == "manter igual qt"').index), 'unit_kg'
] = df.loc[df.index.isin(df_correction.index), 'qt']

df.loc[
    df.index.isin(df_correction.query('correcao != "manter igual qt" and item.str.contains("SALSICHA, DIVERSOS SABORES, COM OU SEM MOLHO")').index), 'unit_kg'
] = df.loc[df.index.isin(df_correction.index), 'valor'] / median_1

df.loc[
    df.index.isin(df_correction.query('correcao != "manter igual qt" and item.str.contains("SALSICHA EM CONSERVA")').index), 'unit_kg'
] = df.loc[df.index.isin(df_correction.index), 'valor'] / median_2


df.loc[df.index.isin(df_correction.index), 'price_kg'] = df.loc[df.index.isin(df_correction.index), 'valor'] / df.loc[df.index.isin(df_correction.index), 'unit_kg']


In [5]:
df.reset_index().to_parquet('data/banco_carnes_un_v10_ed.parquet')